In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
import hpbandster.visualization as hpvis


from hpbandster.optimizers import BOHB as BOHB
import pickle

import numpy as np

In [2]:
import sys
sys.path.insert(0, '..')


import System as SEQ
%run rod_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
test_gridObj = SEQ.Grid(2,{'x':0,'t':1},[[[0,2],[0,10]]],10)
configspecs = {
    'denspt':[5,12],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator': ['elu','relu','selu','swish',
                 'tanh','Addons>gelu','Addons>mish',
                          'Addons>softshrink'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000],
    'initial_lr':[1e-4,1e-2]
}




In [5]:
def u_n(n,x,t):
  mu_n = (2*n+1)*np.pi/4
  T_n =  np.exp(-t*(mu_n**2))
  X_n = -np.cos(mu_n*x)*2.5/(mu_n**2)
  return X_n*T_n

depth = 100

u_pred_Simb = (np.sum((np.array([u_n(n,test_gridObj.grid[:,0],test_gridObj.grid[:,1]) for n in range(depth)])),axis=0)+5)[:,None]
valData = np.concatenate((test_gridObj.grid,u_pred_Simb),axis=1)

In [6]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='rod_GridSearch_1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.

numWorkers = 5
experiment_name = 'rod_experiment'

workers=[]
for i in range(1,numWorkers+1):
    
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=False,
    experiment_name = experiment_name,
    nameserver='127.0.0.1',
    run_id='rod_GridSearch_1',id=i)
    
    w.run(background=True)
    
    workers.append(w)




13:56:29 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x17951b10408; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
13:56:29 WORKER: No dispatcher found. Waiting for one to initiate contact.
13:56:29 WORKER: start listening for jobs
13:56:29 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x17952f70448; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
13:56:29 WORKER: No dispatcher found. Waiting for one to initiate contact.
13:56:29 WORKER: start listening for jobs
13:56:29 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x17952f81b48; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
13:56:29 WORKER: No dispatcher found. Waiting for one to initiate contact.
13:56:29 WORKER: start listening for jobs
13:56:29 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x17952f81688; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
13:56:29 WORKER: No dispatcher found. Waiting for one to initiate contact.
13:56:29 WORKER: start listening for j

Creating client
New experiment. Creating instance.
Applying validation data to test grid
Saving PDE and config
Worker ready
Creating client
Existing experiment. Recovering id.
Applying validation data to test grid
Saving PDE and config
Worker ready
Creating client
Existing experiment. Recovering id.
Applying validation data to test grid
Saving PDE and config
Worker ready
Creating client
Existing experiment. Recovering id.
Applying validation data to test grid
Saving PDE and config
Worker ready
Creating client
Existing experiment. Recovering id.
Applying validation data to test grid
Saving PDE and config
Worker ready


13:56:29 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x17953126ac8; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
13:56:29 WORKER: No dispatcher found. Waiting for one to initiate contact.
13:56:29 WORKER: start listening for jobs


In [7]:
# bohb = BOHB(  configspace = w.get_configspace(),
#               run_id = 'example1', nameserver='127.0.0.1',
#               min_budget=10, max_budget=1000
#            )
# res = bohb.run(n_iterations=25
#               )

bohb = BOHB(  configspace = w.get_configspace(),
              run_id = 'rod_GridSearch_1',
              min_budget=100, max_budget=3000, num_samples = 10
           )


13:56:29 DISPATCHER: started the 'discover_worker' thread
13:56:29 DISPATCHER: started the 'job_runner' thread


In [8]:
bohb.__dict__

{'working_directory': '.',
 'logger': <Logger hpbandster (DEBUG)>,
 'result_logger': None,
 'config_generator': <hpbandster.optimizers.config_generators.bohb.BOHB at 0x1795313f488>,
 'time_ref': None,
 'iterations': [],
 'jobs': [],
 'num_running_jobs': 0,
 'job_queue_sizes': (-1, 0),
 'user_job_queue_sizes': (-1, 0),
 'dynamic_queue_size': True,
 'warmstart_iteration': [],
 'thread_cond': <Condition(<unlocked _thread.RLock object owner=0 count=0 at 0x0000017951AF3630>, 0)>,
 'config': {'time_ref': None,
  'eta': 3,
  'min_budget': 100,
  'max_budget': 3000,
  'budgets': array([ 111.11111111,  333.33333333, 1000.        , 3000.        ]),
  'max_SH_iter': 4,
  'min_points_in_model': None,
  'top_n_percent': 15,
  'num_samples': 10,
  'random_fraction': 0.3333333333333333,
  'bandwidth_factor': 3,
  'min_bandwidth': 0.001},
 'dispatcher': <hpbandster.core.dispatcher.Dispatcher at 0x1795313f2c8>,
 'dispatcher_thread': <Thread(Thread-30, started 9644)>,
 'eta': 3,
 'min_budget': 100,
 'ma

13:56:29 DISPATCHER: Pyro daemon running on localhost:53133
13:56:29 DISPATCHER: Starting worker discovery
13:56:29 DISPATCHER: Found 5 potential workers, 0 currently in the pool.
13:56:29 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.12496
13:56:29 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.22496


In [9]:
res = bohb.run(n_iterations=8, min_n_workers=numWorkers)


13:56:29 wait_for_workers trying to get the condition
13:56:29 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.32496
13:56:29 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.42496
13:56:29 DISPATCHER: discovered new worker, hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.52496
13:56:29 HBMASTER: number of workers changed to 5
13:56:29 DISPATCHER: jobs to submit = 0, number of idle workers = 5 -> waiting!
13:56:29 Enough workers to start this run!
13:56:29 adjust_queue_size: lock accquired
13:56:29 HBMASTER: adjusted queue size to (4, 5)
13:56:29 DISPATCHER: Finished worker discovery
13:56:29 DISPATCHER: Trying to submit another job.
13:56:29 DISPATCHER: jobs to submit = 0, number of idle workers = 5 -> waiting!
13:56:29 HBMASTER: starting run at 1648299389.7113676
13:56:29 start sampling a new configuration.
13:56:29 done sampling a new configuration.
13:56:29 HBMASTER: schedule


Total samples: 2420 
Batch size: 2420 
Total batches: 1 


Total samples: 2420 
Batch size: 2420 
Total batches: 1 


Total samples: 500 
Batch size: 500 
Total batches: 1 


Total samples: 980 
Batch size: 980 
Total batches: 1 


Total samples: 2880 
Batch size: 2880 
Total batches: 1 



13:57:29 DISPATCHER: Starting worker discovery
13:57:29 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
13:57:29 DISPATCHER: Finished worker discovery



Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.004118545912206173.

Epoch 00062: ReduceLROnPlateau reducing learning rate to 0.0020592729561030865.

Epoch 00090: ReduceLROnPlateau reducing learning rate to 0.0010296364780515432.


13:57:49 WORKER: done with job (0, 0, 4), trying to register it.
13:57:49 DISPATCHER: job (0, 0, 4) finished
13:57:49 WORKER: registered result for job (0, 0, 4) with dispatcher
13:57:49 DISPATCHER: register_result: lock acquired
13:57:49 DISPATCHER: job (0, 0, 4) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.22496 finished
13:57:49 job_id: (0, 0, 4)
kwargs: {'config': {'activator': 'Addons>mish', 'batch_size': 5000, 'denspt': 11, 'initial_lr': 0.008237092039893007, 'loss': 'mse', 'numLayers': 20, 'numNeurons': 52, 'optimizer': 'RMSprop'}, 'budget': 111.1111111111111, 'working_directory': '.'}
result: {'loss': 5.912182831454428, 'info': {'L1': 5.912182831454428, 'L2': 2.62150661222862, 'MAX': 0.8229426113219738, 'TrainTime': 129.609375}}
exception: None

13:57:49 job_callback for (0, 0, 4) started
13:57:49 DISPATCHER: Trying to submit another job.
13:57:49 job_callback for (0, 0, 4) got condition
13:57:49 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> w

...

15:09:37 DISPATCHER: Starting worker discovery
15:09:37 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
15:09:37 DISPATCHER: Finished worker discovery
15:10:16 WORKER: done with job (6, 0, 4), trying to register it.
15:10:16 WORKER: registered result for job (6, 0, 4) with dispatcher
15:10:16 DISPATCHER: job (6, 0, 4) finished
15:10:16 DISPATCHER: register_result: lock acquired
15:10:16 DISPATCHER: job (6, 0, 4) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.42496 finished
15:10:16 job_id: (6, 0, 4)
kwargs: {'config': {'activator': 'Addons>gelu', 'batch_size': 15000, 'denspt': 11, 'initial_lr': 0.0013312913939173394, 'loss': 'mse', 'numLayers': 15, 'numNeurons': 11, 'optimizer': 'Nadam'}, 'budget': 3000.0, 'working_directory': '.'}
result: {'loss': 0.2459338859758297, 'info': {'L1': 0.2459338859758297, 'L2': 0.0072262707099517084, 'MAX': 0.24815643895798623, 'TrainTime': 2683.8125}}
exception: None

15:10:16 job_callback for (6, 0, 4) started
15:10:16 D


Epoch 01928: ReduceLROnPlateau reducing learning rate to 0.0019607136491686106.


15:11:37 DISPATCHER: Starting worker discovery
15:11:37 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
15:11:37 DISPATCHER: Finished worker discovery
15:12:37 DISPATCHER: Starting worker discovery
15:12:37 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
15:12:37 DISPATCHER: Finished worker discovery



Epoch 02650: ReduceLROnPlateau reducing learning rate to 0.003134208032861352.

Epoch 02527: ReduceLROnPlateau reducing learning rate to 0.0009803568245843053.


15:13:37 DISPATCHER: Starting worker discovery
15:13:37 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
15:13:37 DISPATCHER: Finished worker discovery
15:14:37 DISPATCHER: Starting worker discovery
15:14:37 DISPATCHER: Found 5 potential workers, 5 currently in the pool.
15:14:37 DISPATCHER: Finished worker discovery
15:14:59 WORKER: done with job (7, 0, 2), trying to register it.
15:14:59 WORKER: registered result for job (7, 0, 2) with dispatcher
15:14:59 DISPATCHER: job (7, 0, 2) finished
15:14:59 DISPATCHER: register_result: lock acquired
15:14:59 DISPATCHER: job (7, 0, 2) on hpbandster.run_rod_GridSearch_1.worker.DESKTOP-NKTB8UL.10312.52496 finished
15:14:59 job_id: (7, 0, 2)
kwargs: {'config': {'activator': 'Addons>gelu', 'batch_size': 15000, 'denspt': 8, 'initial_lr': 0.006268416082058757, 'loss': 'mae', 'numLayers': 24, 'numNeurons': 64, 'optimizer': 'Adam'}, 'budget': 3000.0, 'working_directory': '.'}
result: {'loss': 2.3015270735211084, 'info': {'L1': 2.3015270

In [12]:
np.save('HpBandster_Results',res,allow_pickle=True)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

15:16:07 HBMASTER: shutdown initiated, shutdown_workers = True
15:16:08 WORKER: shutting down now!
15:16:08 WORKER: shutting down now!
15:16:08 WORKER: shutting down now!
15:16:08 WORKER: shutting down now!
15:16:08 WORKER: shutting down now!


In [32]:
res.__dict__['data'][res.get_incumbent_id()]


config:{'activator': 'elu', 'batch_size': 15000, 'denspt': 7, 'initial_lr': 0.008136987243859039, 'loss': 'mse', 'numLayers': 8, 'numNeurons': 42, 'optimizer': 'Adam'}
config_info:
{}
333.3333333333333: 0.33812513189675153	losses:
	1000.0: 0.6484396094324323	losses:
	3000.0: 0.06484265556770885	time stamps: {333.3333333333333: {'submitted': 2517.8033833503723, 'started': 2517.819383621216, 'finished': 2569.4526178836823}, 1000.0: {'submitted': 2918.9522185325623, 'started': 2919.0231952667236, 'finished': 3041.411642074585}, 3000.0: {'submitted': 3055.3004376888275, 'started': 3055.337425470352, 'finished': 3460.675125360489}}